In [1]:
import os
import numpy as np
import importlib
import pandas as pd
import importlib

os.environ["CUDA_VISIBLE_DEVICES"] = "-1" # This is to disable GPU
os.environ["TF_ENABLE_ONEDNN_OPTS"] = "0"

import prepare_ml_dataset
import train_nn_model

importlib.reload(prepare_ml_dataset)
importlib.reload(train_nn_model)

2025-02-27 16:50:30.589926: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-27 16:50:30.657064: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-27 16:50:30.720414: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740693030.769872  943058 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740693030.784320  943058 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-27 16:50:30.915187: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

<module 'train_nn_model' from '/home/jliao/workspace/GitHub/ml_ringcurrent_ion/train_nn_model.py'>

In [2]:
energy = '51767680'
species = 'h'

In [ ]:
directories, dataset_csv, data_settings = prepare_ml_dataset.initializ_var(energy, species)

x_train, x_valid, x_test, y_train, y_valid, y_test = prepare_ml_dataset.prepare_ml_dataset(energy, species, recalc=True)

print(x_train.shape, x_valid.shape, x_test.shape, y_train.shape, y_valid.shape, y_test.shape)

Reading csv data for probeb

/home/jliao/anaconda3/envs/notebook/lib/python3.11/site-packages/dask/utils.py:1226: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  return getattr(__obj, self.method)(*args, **kwargs)
/home/jliao/anaconda3/envs/notebook/lib/python3.11/site-packages/dask/utils.py:1226: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  return getattr(__obj, self.method)(*args, **kwargs)
/home/jliao/anaconda3/envs/notebook/lib/python3.11/site-packages/dask/utils.py:1226: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  return getattr(__obj, self.method)(*args, **kwargs)
/home/jliao/anaconda3/envs/notebook/lib/python3.1

7.868543463017787 1.040252310912895 4.4543978869653404 6.8282911521048915
19.92544708333316 -19.85855629142716 0.033445395953000556 39.78400337476032
78.0 -232.8000030517578 -77.4000015258789 310.8000030517578
348.0 0.0 174.0 348.0
379.0 0.0 189.5 379.0
2471.39990234375 3.400000095367432 1237.3999512195587 2467.9999022483826
255.0 64.5999984741211 159.79999923706055 190.4000015258789
8.3 0.0 4.15 8.3
56.76599884033203 0.050000000745058 28.407999420538545 56.71599883958697
72.12999725341797 0.1359999924898148 36.13299862295389 71.99399726092815
927.0999755859376 243.6999969482422 585.3999862670898 683.3999786376954
32.51399993896484 -27.52799987792969 2.493000030517573 60.04199981689453
31.9219970703125 -38.81400299072266 -3.4460029602050817 70.73600006103516


/home/jliao/workspace/GitHub/ml_ringcurrent_ion/prepare_ml_dataset.py:307: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_full['episodes'] = df_full['time'].apply(lambda x: math.floor((x-t0)/episode_time))


475943 119522 116124


In [ ]:
para_name = "learning_rate"
para_set = [1.e-4, 1.5e-3, 1.e-3]

final_train_loss = np.zeros(len(para_set))
final_valid_loss = np.zeros(len(para_set))
total_history = dict()
valid_r2s = np.zeros(len(para_set))

for ipara in range(len(para_set)):
    parameter = para_set[ipara]

    model, history, valid_r2 = train_nn_model.nn_model(x_train, y_train, x_valid, y_valid, data_settings["y_name_log"], 
                                                       output_dir = directories["training_output_dir"] , n_neurons = 18, patience = 32,
                                                       learning_rate = parameter, epochs = 10, batch_size = 8)
    
    total_history[str(parameter)] = history.history
    final_train_loss[ipara] = history.history['loss'][-1]
    final_valid_loss[ipara] = history.history['val_loss'][-1]
    valid_r2s[ipara] = valid_r2